In [6]:
import pandas as pd
import plotly.graph_objects as go
from dash import Dash, dcc, html, Input, Output
import textwrap

# -------------------------------
# Configuration
# -------------------------------
FILE = r"c:\NBDB/PCTWIN_PRJ_Tracking/INCOIS_PCTWIN_tracking.xlsx"
PROJECT_START = pd.to_datetime("2024-02-01")
TICK_SPACING_DAYS = 90  # ~3 months
WRAP_WIDTH = 40  # characters for hovertext wrapping

# -------------------------------
# Helper functions
# -------------------------------
def convert_to_date(m_str):
    if pd.isna(m_str):
        return pd.NaT
    try:
        s = str(m_str).strip().upper().replace("M", "").replace(",", "")
        m = int(s)
        return PROJECT_START + pd.DateOffset(months=m - 1)
    except Exception:
        return pd.NaT

def lighten_color(rgb_str, factor=0.5):
    import re
    match = re.match(r"rgba\((\d+),(\d+),(\d+),([\d.]+)\)", rgb_str)
    if not match:
        return rgb_str
    r, g, b, a = map(float, match.groups())
    r = r + (255 - r) * factor
    g = g + (255 - g) * factor
    b = b + (255 - b) * factor
    return f"rgba({int(r)},{int(g)},{int(b)},{a})"

def wrap_text(text):
    return "<br>".join(textwrap.wrap(str(text), WRAP_WIDTH))

# -------------------------------
# Load data
# -------------------------------
df = pd.read_excel(FILE)
df["Start"] = df["Start"].apply(convert_to_date)
df["End"] = df["End"].apply(convert_to_date)
df["Sub Tasks"] = df["Sub Tasks"].fillna("")

df["Start_days"] = (df["Start"] - PROJECT_START).dt.days
df["End_days"] = (df["End"] - PROJECT_START).dt.days

# -------------------------------
# Dash state
# -------------------------------
expanded_tasks = set()
color_map = {}  # Will assign colors to institutes
base_colors = [
    "rgba(205,133,63,0.8)",   # Peru
    "rgba(46,139,87,0.8)",    # Sea Green
    "rgba(112,128,144,0.8)",  # Slate Gray
    "rgba(244,164,96,0.8)",   # Sandy Brown
    "rgba(70,130,180,0.8)"    # Steel Blue
]
# -------------------------------
# Build Gantt function
# -------------------------------
def build_gantt():
    fig = go.Figure()
    y_labels = []
    y_positions = []
    y_counter = 0
    used_legend = set()  # for unique legend entries
    color_index = 0

    for task_number in df["Task Numbers"].unique():
        # --- Main task: first row where Sub Tasks is empty ---
        main_row = df[(df["Task Numbers"] == task_number) & (df["Sub Tasks"] == "")].head(1)
        if main_row.empty:
            continue
        main_row = main_row.iloc[0]
        if pd.isna(main_row["Start_days"]) or pd.isna(main_row["End_days"]):
            continue

        resp_list = [r.strip() for r in str(main_row["Responsible"]).split(",")]
        leader_list = [r.strip() for r in str(main_row["Task Leader"]).split(",")]

        # Assign colors for new institutes
        for r in resp_list:
            if r not in color_map:
                color_map[r] = base_colors[color_index % len(base_colors)]
                color_index += 1

        # Split bar width for multiple responsible institutes
        total_width = main_row["End_days"] - main_row["Start_days"]
        seg_width = total_width / len(resp_list)
        start_day = main_row["Start_days"]

        # --- Main task bars ---
        for i, r in enumerate(resp_list):
            col = color_map[r]
            show_legend = r not in used_legend
            fig.add_trace(go.Bar(
                x=[seg_width],
                y=[y_counter],
                base=start_day + i * seg_width,
                orientation="h",
                marker=dict(color=col),
                hovertemplate=(
                    f"<b>Task Number:</b> {main_row['Task Numbers']}<br>"
                    f"<b>Deliverable:</b> {wrap_text(main_row['Main Tasks'])}<br>"
                    f"<b>Responsible:</b> {', '.join(resp_list)}<br>"
                    f"<b>Task Leader:</b> {', '.join(leader_list)}<br>"
                    f"<b>Duration:</b> {main_row['Start'].strftime('%b-%Y')} → {main_row['End'].strftime('%b-%Y')}<extra></extra>"
                ),
                customdata=[task_number],
                name=r if show_legend else None,
                showlegend=show_legend
            ))
            used_legend.add(r)

        y_labels.append(str(task_number))
        y_positions.append(y_counter)
        y_counter += 1

        # --- Subtasks (only if expanded) ---
        if task_number in expanded_tasks:
            sub_rows = df[(df["Task Numbers"] == task_number) & (df["Sub Tasks"] != "")]
            for idx, (_, row) in enumerate(sub_rows.iterrows(), start=1):
                if pd.isna(row["Start_days"]) or pd.isna(row["End_days"]):
                    continue
                sub_resp_list = [r.strip() for r in str(row["Responsible"]).split(",")]
                total_sub_width = row["End_days"] - row["Start_days"]
                seg_width_sub = total_sub_width / len(sub_resp_list)
                start_sub = row["Start_days"]

                for j, r in enumerate(sub_resp_list):
                    col = lighten_color(color_map[r], factor=0.5)
                    fig.add_trace(go.Bar(
                        x=[seg_width_sub],
                        y=[y_counter],
                        base=start_sub + j * seg_width_sub,
                        orientation="h",
                        marker=dict(color=col),
                        hovertemplate=(
                            f"<b>Task Number:</b> {row['Task Numbers']}<br>"
                            f"<b>Deliverable:</b> {wrap_text(row['Main Tasks'])}<br>"
                            f"<b>Subtask:</b> {wrap_text(row['Sub Tasks'])}<br>"
                            f"<b>Responsible:</b> {row['Responsible']}<br>"
                            f"<b>Task Leader:</b> {row['Task Leader']}<br>"
                            f"<b>Duration:</b> {row['Start'].strftime('%b-%Y')} → {row['End'].strftime('%b-%Y')}<extra></extra>"
                        ),
                        customdata=[task_number],
                        name=None,  # subtasks not in legend
                        showlegend=False
                    ))
                y_labels.append(f"SubT-{idx}")
                y_positions.append(y_counter)
                y_counter += 1

    # --- X-axis ticks ---
    max_days = int(df["End_days"].max()) + 30
    tick_days = list(range(0, max_days + 1, TICK_SPACING_DAYS))
    tick_text = [(PROJECT_START + pd.Timedelta(days=d)).strftime("%b-%Y") for d in tick_days]

    fig.update_layout(
        barmode="overlay",
        yaxis=dict(
            tickvals=y_positions,
            ticktext=y_labels,
            autorange="reversed"
        ),
        xaxis=dict(
            tickvals=tick_days,
            ticktext=tick_text,
            tickangle=45,
            title="Month-Year"
        ),
        showlegend=True,
        height=700,
        title="Interactive PCTWIN Gantt Chart with Subtasks"
    )
    fig.update_layout(
    legend=dict(
        orientation="h",   # make horizontal
        yanchor="bottom",
        y=1.02,            # move just above plot area
        xanchor="left",
        x=0
    )
)
    return fig

# -------------------------------
# Dash App
# -------------------------------
app = Dash(__name__)
app.layout = html.Div([
    html.H3("Interactive Gantt Chart"),
    dcc.Graph(id="gantt", figure=build_gantt())
])

@app.callback(
    Output("gantt", "figure"),
    Input("gantt", "clickData")
)
def toggle_subtasks(clickData):
    global expanded_tasks
    if clickData:
        clicked_task_number = clickData["points"][0]["customdata"]
        if clicked_task_number in expanded_tasks:
            expanded_tasks.remove(clicked_task_number)
        else:
            expanded_tasks.add(clicked_task_number)
    return build_gantt()

if __name__ == "__main__":
    app.run(debug=True)
